In [ ]:
# https://colab.research.google.com/drive/13EoKtPz0LLhOfAfPQoue_HylFR9hl6Ba?usp=sharing
# https://www.kaggle.com/code/shawkyelgendy/pump-sensor-data-timeseriesanalysis/input?select=sensor.csv

# Importing the required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve , auc
from sklearn.metrics import r2_score,mean_squared_error
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [ ]:
df = pd.read_csv("/content/sensor.csv")
df


In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# Cleaning & Pre-processing data

df.isnull().sum()

Unnamed: 0             0
timestamp              0
sensor_00          10208
sensor_01            369
sensor_02             19
sensor_03             19
sensor_04             19
sensor_05             19
sensor_06           4798
sensor_07           5451
sensor_08           5107
sensor_09           4595
sensor_10             19
sensor_11             19
sensor_12             19
sensor_13             19
sensor_14             21
sensor_15         220320
sensor_16             31
sensor_17             46
sensor_18             46
sensor_19             16
sensor_20             16
sensor_21             16
sensor_22             41
sensor_23             16
sensor_24             16
sensor_25             36
sensor_26             20
sensor_27             16
sensor_28             16
sensor_29             72
sensor_30            261
sensor_31             16
sensor_32             68
sensor_33             16
sensor_34             16
sensor_35             16
sensor_36             16
sensor_37             16


In [ ]:
# As it is clearly seen that the Sensor_15 has all null valuee. So droping it will not have any impact on the final result.
df.drop('sensor_15',axis = 1, inplace = True)
df.head()

,Unnamed: 0,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,...,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,machine_status
0,0,2018-04-01 00:00:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
1,1,2018-04-01 00:01:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
2,2,2018-04-01 00:02:00,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,...,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,NORMAL
3,3,2018-04-01 00:03:00,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,...,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,NORMAL
4,4,2018-04-01 00:04:00,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,...,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,NORMAL


In [ ]:
#In addition to this it is considered that the unamed,timestamp columns dont have any impact on operating condition of pump
df.drop(['Unnamed: 0','timestamp'],axis=1,inplace= True)
df.head()

,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,...,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,machine_status
0,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
1,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
2,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,15.61777,15.01013,...,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,NORMAL
3,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,15.69734,15.08247,...,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,NORMAL
4,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,15.69734,15.08247,...,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,NORMAL


In [ ]:
# Focusing on the target variable :
df['machine_status'].value_counts()

NORMAL        205836
RECOVERING     14477
BROKEN             7
Name: machine_status, dtype: int64

In [ ]:
# Before perfoming an operation we will create a copy of the dataset.
df_a = df
df_a['machine_status'].replace('RECOVERING','ABNORMAL',inplace = True)
df_a['machine_status'].replace('BROKEN','ABNORMAL',inplace =True)
df_a['machine_status'].value_counts()

NORMAL      205836
ABNORMAL     14484
Name: machine_status, dtype: int64

In [ ]:
df_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220320 entries, 0 to 220319
Data columns (total 52 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   sensor_00       210112 non-null  float64
 1   sensor_01       219951 non-null  float64
 2   sensor_02       220301 non-null  float64
 3   sensor_03       220301 non-null  float64
 4   sensor_04       220301 non-null  float64
 5   sensor_05       220301 non-null  float64
 6   sensor_06       215522 non-null  float64
 7   sensor_07       214869 non-null  float64
 8   sensor_08       215213 non-null  float64
 9   sensor_09       215725 non-null  float64
 10  sensor_10       220301 non-null  float64
 11  sensor_11       220301 non-null  float64
 12  sensor_12       220301 non-null  float64
 13  sensor_13       220301 non-null  float64
 14  sensor_14       220299 non-null  float64
 15  sensor_16       220289 non-null  float64
 16  sensor_17       220274 non-null  float64
 17  sensor_18 

In [ ]:
# As it is clearly seen that the target variable is of the dtype object we need to convert it.
labelencoder_X = LabelEncoder()
df_a.iloc[:,51] = labelencoder_X.fit_transform(df_a.iloc[:,51])

<ipython-input-34-168e7646cb46>:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_a.iloc[:,51] = labelencoder_X.fit_transform(df_a.iloc[:,51])


In [ ]:
df_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220320 entries, 0 to 220319
Data columns (total 52 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   sensor_00       210112 non-null  float64
 1   sensor_01       219951 non-null  float64
 2   sensor_02       220301 non-null  float64
 3   sensor_03       220301 non-null  float64
 4   sensor_04       220301 non-null  float64
 5   sensor_05       220301 non-null  float64
 6   sensor_06       215522 non-null  float64
 7   sensor_07       214869 non-null  float64
 8   sensor_08       215213 non-null  float64
 9   sensor_09       215725 non-null  float64
 10  sensor_10       220301 non-null  float64
 11  sensor_11       220301 non-null  float64
 12  sensor_12       220301 non-null  float64
 13  sensor_13       220301 non-null  float64
 14  sensor_14       220299 non-null  float64
 15  sensor_16       220289 non-null  float64
 16  sensor_17       220274 non-null  float64
 17  sensor_18 

In [ ]:
# Now we need to check fot the null values in each column
for  col in df_a.columns:
    print(col,df_a[col].isna().sum())

sensor_00 10208
sensor_01 369
sensor_02 19
sensor_03 19
sensor_04 19
sensor_05 19
sensor_06 4798
sensor_07 5451
sensor_08 5107
sensor_09 4595
sensor_10 19
sensor_11 19
sensor_12 19
sensor_13 19
sensor_14 21
sensor_16 31
sensor_17 46
sensor_18 46
sensor_19 16
sensor_20 16
sensor_21 16
sensor_22 41
sensor_23 16
sensor_24 16
sensor_25 36
sensor_26 20
sensor_27 16
sensor_28 16
sensor_29 72
sensor_30 261
sensor_31 16
sensor_32 68
sensor_33 16
sensor_34 16
sensor_35 16
sensor_36 16
sensor_37 16
sensor_38 27
sensor_39 27
sensor_40 27
sensor_41 27
sensor_42 27
sensor_43 27
sensor_44 27
sensor_45 27
sensor_46 27
sensor_47 27
sensor_48 27
sensor_49 27
sensor_50 77017
sensor_51 15383
machine_status 0


Droping The Null Values Case 1

In [ ]:
# Considering the nature of our data set we cant' fill the null values with mean/mode of data.
# We will first try and implement a model by dropping all the null values.
df_b = df_a.dropna()

In [ ]:
for  col in df_b.columns:
    print(col,df_b[col].isna().sum())

sensor_00 0
sensor_01 0
sensor_02 0
sensor_03 0
sensor_04 0
sensor_05 0
sensor_06 0
sensor_07 0
sensor_08 0
sensor_09 0
sensor_10 0
sensor_11 0
sensor_12 0
sensor_13 0
sensor_14 0
sensor_16 0
sensor_17 0
sensor_18 0
sensor_19 0
sensor_20 0
sensor_21 0
sensor_22 0
sensor_23 0
sensor_24 0
sensor_25 0
sensor_26 0
sensor_27 0
sensor_28 0
sensor_29 0
sensor_30 0
sensor_31 0
sensor_32 0
sensor_33 0
sensor_34 0
sensor_35 0
sensor_36 0
sensor_37 0
sensor_38 0
sensor_39 0
sensor_40 0
sensor_41 0
sensor_42 0
sensor_43 0
sensor_44 0
sensor_45 0
sensor_46 0
sensor_47 0
sensor_48 0
sensor_49 0
sensor_50 0
sensor_51 0
machine_status 0


In [ ]:

df_b.head()

,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,...,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,machine_status
0,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,1
1,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,1
2,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,15.61777,15.01013,...,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,1
3,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,15.69734,15.08247,...,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,1
4,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,15.69734,15.08247,...,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,1


In [ ]:
scaler = MinMaxScaler()
df_std = scaler.fit_transform(df_b.iloc[:,:51])
df_std = pd.DataFrame(df_std)
df_std.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,0.967194,0.739583,0.873046,0.872093,0.943770,0.764598,0.626734,0.683630,0.638905,0.602026,...,0.019658,0.047664,0.057143,0.181429,0.069791,0.04363,0.292178,0.186992,0.221197,0.178327
1,0.967194,0.739583,0.873046,0.872093,0.943770,0.764598,0.626734,0.683630,0.638905,0.602026,...,0.019658,0.047664,0.057143,0.181429,0.069791,0.04363,0.292178,0.186992,0.221197,0.178327
2,0.959089,0.747396,0.873046,0.877907,0.950518,0.735461,0.622673,0.679645,0.640988,0.600290,...,0.019658,0.046711,0.055639,0.180000,0.070788,0.04363,0.287653,0.184282,0.219410,0.180709
3,0.965264,0.739583,0.871093,0.877907,0.934427,0.769891,0.622335,0.688535,0.644259,0.603184,...,0.018803,0.043851,0.054135,0.177143,0.070788,0.04363,0.287653,0.182927,0.218517,0.180113
4,0.959475,0.740885,0.873046,0.877907,0.946884,0.765891,0.624027,0.687002,0.644259,0.603184,...,0.018803,0.045758,0.052632,0.178571,0.072782,0.04712,0.292825,0.181572,0.220304,0.178327


In [ ]:
df_std.describe()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
count,119103.000000,119103.000000,119103.000000,119103.000000,119103.000000,119103.000000,119103.000000,119103.000000,119103.000000,119103.000000,...,119103.000000,119103.000000,119103.000000,119103.000000,119103.000000,119103.000000,119103.000000,119103.000000,119103.000000,119103.000000
mean,0.921473,0.766555,0.799355,0.694445,0.913302,0.751116,0.617699,0.670541,0.621915,0.593828,...,0.034194,0.054237,0.062300,0.064836,0.051080,0.076221,0.206331,0.121762,0.174544,0.172892
std,0.176496,0.070457,0.130480,0.129773,0.128936,0.147084,0.077619,0.073496,0.068577,0.066421,...,0.027546,0.032545,0.037113,0.048961,0.040468,0.049966,0.126110,0.058382,0.053160,0.079778
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.953300,0.731771,0.767578,0.616279,0.932697,0.727034,0.620981,0.665543,0.620464,0.600290,...,0.023932,0.039085,0.040602,0.041429,0.030907,0.048866,0.109890,0.092141,0.153915,0.154808
50%,0.962949,0.768229,0.822265,0.703488,0.940310,0.772455,0.629442,0.683630,0.627602,0.603184,...,0.032479,0.050524,0.057143,0.055714,0.043868,0.068063,0.177763,0.109756,0.176541,0.172373
75%,0.970668,0.804687,0.873046,0.784883,0.947403,0.815661,0.636886,0.687002,0.638905,0.604631,...,0.040171,0.062917,0.075188,0.072857,0.058824,0.090750,0.281836,0.134146,0.202441,0.190533
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Regression Model 1

In [ ]:
# Model Implemention on Case -1
X1=df_std
X1

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,0.967194,0.739583,0.873046,0.872093,0.943770,0.764598,0.626734,0.683630,0.638905,0.602026,...,0.019658,0.047664,0.057143,0.181429,0.069791,0.043630,0.292178,0.186992,0.221197,0.178327
1,0.967194,0.739583,0.873046,0.872093,0.943770,0.764598,0.626734,0.683630,0.638905,0.602026,...,0.019658,0.047664,0.057143,0.181429,0.069791,0.043630,0.292178,0.186992,0.221197,0.178327
2,0.959089,0.747396,0.873046,0.877907,0.950518,0.735461,0.622673,0.679645,0.640988,0.600290,...,0.019658,0.046711,0.055639,0.180000,0.070788,0.043630,0.287653,0.184282,0.219410,0.180709
3,0.965264,0.739583,0.871093,0.877907,0.934427,0.769891,0.622335,0.688535,0.644259,0.603184,...,0.018803,0.043851,0.054135,0.177143,0.070788,0.043630,0.287653,0.182927,0.218517,0.180113
4,0.959475,0.740885,0.873046,0.877907,0.946884,0.765891,0.624027,0.687002,0.644259,0.603184,...,0.018803,0.045758,0.052632,0.178571,0.072782,0.047120,0.292825,0.181572,0.220304,0.178327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119098,0.970668,0.680989,0.812500,0.880814,0.947057,0.850438,0.643655,0.679645,0.623141,0.604631,...,0.049573,0.053384,0.057143,0.067143,0.095713,0.078534,0.318681,0.120596,1.000000,0.179815
119099,0.971440,0.680990,0.812500,0.880814,0.924911,0.841146,0.649408,0.688535,0.634146,0.603184,...,0.048718,0.058151,0.057143,0.067143,0.093719,0.085515,0.318681,0.120596,1.000000,0.178029
119100,0.968352,0.682292,0.810546,0.880814,0.931659,0.849958,0.650761,0.683630,0.630875,0.604631,...,0.047863,0.061964,0.057143,0.065714,0.088734,0.092496,0.314156,0.120596,1.000000,0.177731
119101,0.972598,0.680990,0.812500,0.880814,0.940137,0.856715,0.643993,0.688535,0.614217,0.602026,...,0.048718,0.064824,0.060150,0.067143,0.083749,0.101222,0.302521,0.120596,1.000000,0.176838


In [ ]:
Y1=df_b['machine_status']
Y1

0         1
1         1
2         1
3         1
4         1
         ..
143319    1
143320    1
143321    1
143322    1
143323    1
Name: machine_status, Length: 119103, dtype: int64

In [ ]:
# Splitting the data
X1_train,X1_test,Y1_train,Y1_test=train_test_split(X1,Y1,test_size=0.3)

In [ ]:
# Logistic Regression Model
LR = LogisticRegression(C = 0.2)
model1 = LR.fit(X1_train,Y1_train)
Y_pred1= model1.predict(X1_test)


In [ ]:
print("The confusion matrix for Logistic Regression is :\n" ,confusion_matrix(Y1_test,Y_pred1))
print(' The RMSE score for Logistic Regression is : \n',np.sqrt(mean_squared_error(Y1_test , Y_pred1)))
print(' The R2 score for Logistic Regression is :\n',r2_score(Y1_test , Y_pred1))
# print('The roc_auc_score is: \n',roc_auc_score(Y1_test,Y_pred1))

The confusion matrix for Logistic Regression is :
 [[  874    50]
 [   79 34728]]
 The RMSE score for Logistic Regression is : 
 0.060085858359056665
 The R2 score for Logistic Regression is :
 0.8566834593280424


Decision Tree Model -2

In [ ]:
DT = DecisionTreeClassifier(max_depth =2 )
model2= DT.fit(X1_train,Y1_train)
Y_pred2 = model2.predict(X1_test)

In [ ]:
print("The confusion matrix for Decision Tree Classifier is :\n" ,confusion_matrix(Y1_test,Y_pred2))
print(' The RMSE score for Decision Tree Classifier is : \n',np.sqrt(mean_squared_error(Y1_test , Y_pred2)))
print(' The R2 score for Decision Tree Classifier is :\n',r2_score(Y1_test , Y_pred2))
# print('The roc_auc_score is: \n',roc_auc_score(Y1_test,Y_pred2))

The confusion matrix for Decision Tree Classifier is :
 [[  845    79]
 [    4 34803]]
 The RMSE score for Decision Tree Classifier is : 
 0.04819660648607607
 The R2 score for Decision Tree Classifier is :
 0.9077885823583528


KNN Classifer MOdel 3

In [ ]:
KNN = KNeighborsClassifier(n_neighbors=4,weights='uniform')
model3 = KNN.fit(X1_train, Y1_train)
y_pred3 = model3.predict(X1_test)

In [ ]:
print(mean_absolute_error(Y1_test, y_pred3))

In [ ]:
print("The confusion matrix for KNN Classifer is :\n" ,confusion_matrix(Y1_test,y_pred3))
print(' The RMSE score for KNN Classifer is : \n',np.sqrt(mean_squared_error(Y1_test , y_pred3)))
print(' The R2 score for KNN Classifer is :\n',r2_score(Y1_test , y_pred3))
# print('The roc_auc_score is: \n',roc_auc_score(Y1_test,y_pred3))

The confusion matrix for KNN Classifer is :
 [[  922     2]
 [    2 34805]]
 The RMSE score for KNN Classifer is : 
 0.0105805296899169
 The R2 score for KNN Classifer is :
 0.9955560762582338
